In [6]:
from operator import index
from textwrap import indent
import akantu as aka
import numpy as np


In [14]:
# Lenght of the bar (m)
L = 50*10**-3  
# Number of linear elements (n_el)
n_el = 5
# Lenght of each linear element (h)
h = L/n_el
# Cross sectional area (m2)
A = 1*10**-3 

# Applied strain rate (s-1)
# strain_rate = 10.0**2
# strain_rate = 10.0**3
strain_rate = 10.0**4
# strain_rate = 10.0**5

# Applied velocity
vel = strain_rate*L/2 


# Limit stress / critical stress (stress_c) (Pa)
stress_c = 300.0*10**6 
# Young's module (Pa)
E = 275.0*10**9  

In [15]:
# Read material file
aka.parseInput('material.dat')

# Read mesh
spatial_dimension = 2
mesh = aka.Mesh(spatial_dimension)
mesh.read('../LOG/bar.msh')

# Create model
model = aka.SolidMechanicsModelCohesive(mesh)
model.initFull(_analysis_method=aka._static, _is_extrinsic=True)


# Configure static solver
solver = model.getNonLinearSolver('static')
solver.set('max_iterations', 100)
solver.set('threshold', 1e-10)
solver.set("convergence_type", aka.SolveConvergenceCriteria.residual)

# Solver (explicit Newmark with lumped mass)
model.initNewSolver(aka._explicit_lumped_mass)
# Dynamic insertion of cohesive elements
model.updateAutomaticInsertion()

In [16]:
# Critical time step
critical_time_step = model.getStableTimeStep()
# Adopted time step
dt = critical_time_step*0.1

# Total time of simulation (s)
time_simulation = 6.0*10**-7

# Number of time steps (n_steps)
n_steps = int(time_simulation/dt)
print(n_steps)

11


In [11]:
class FixedVelocity (aka.DirichletFunctor):
    """
    Fixed velocity at the boundaries
    """

    def __init__(self, axis, vel):
        super().__init__(axis)
        self.axis = axis
        self.time = 0
        self.vel = vel
    
    def set_time(self, t):
        self.time = t
    
    def __call__(self, node, flags, disp, coord):
        flags[int(self.axis)] = True
        disp[int(self.axis)] = self.vel*self.time
        

In [12]:
# Apply BCs

# Apply Dirichlet BC to block dispacements at y direction on top and botton of the elements
model.applyBC(aka.FixedValue(0., aka._y), 'YBlocked')

# Apply constant velocity at the boundaries
functor_left = FixedVelocity(aka._x, -vel)
functor_right = FixedVelocity(aka._x, vel)
model.applyBC(functor_left, 'left')
model.applyBC(functor_right, 'right')



In [13]:
# Initial value 

n_nodes = mesh.getNbNodes()
u0 = model.getDisplacement()
v0 = model.getVelocity()

# Initial velocity profile
v0[:,0] = np.array([strain_rate * x for x,y in mesh.getNodes()])

# Initial displacement (u0) 
if strain_rate < 5.0 * 10.0**3:
    u0[:,0] = np.array([0.98*stress_c*x / E for x,y in mesh.getNodes()])


In [ ]:
# Main algorithm

# Initiation of variables
Epot = np.zeros(n_steps)
Ekin = np.zeros(n_steps)
Edis = np.zeros(n_steps)
Erev = np.zeros(n_steps)
Econ = np.zeros(n_steps)
Wext = np.zeros(n_steps)
work = 0.0
fp_left = 0.0
fp_right = 0.0
avg_stress = np.zeros(n_steps)

for n in range(n_steps):

    # Apply velocity at the boundaries 
    functor_left.set_time(dt*n)
    functor_right.set_time(dt*n)
    model.applyBC(functor_left, 'left')
    model.applyBC(functor_right, 'right')

    # Run simulation
    model.checkCohesiveStress()
    model.solveStep('explicit_lumped')

    # Outputs
    u = model.getDisplacement()[:,0]
    v = model.getVelocity()[:,0]
    a = model.getAcceleration()[:,0]
    fint = model.getInternalForce()[:,0]
    stress = model.getMaterial(0).getStress(aka._triangle_3)
    stress_xx = stress[:,0]
    avg_stress = np.mean(stress_xx)


    # Energy balance

    # External energy
    # Reaction force at the boundaries 
    nodes_left = mesh.getElementGroup('left').getNodeGroup().getNodes()
    nodes_right = mesh.getElementGroup('right').getNodeGroup().getNodes()
    # Current time step (fn)
    fn_left = np.sum(fint[nodes_left])
    fn_right = np.sum(fint[nodes_right])
    # The reaction force (fr) is taken as an average between fn (current time step) and fp (previous time step)
    fr_left = (fn_left+fp_left)*0.5
    fr_right = (fn_right+fp_right)*0.5
    # Stress at the boundary
    stress_bound_left = fr_left / A
    stress_bound_right = fr_right / A
    # External work (Wext)
    Wext = work + stress_bound_left*-vel + stress_bound_right * vel
    work = Wext
    fp_left = fn_left
    fp_right = fn_right
    
    # Get other energies
    Epot[n] = model.getEnergy('potential')
    Ekin[n] = model.getEnergy('kinetic')
    Edis[n] = model.getEnergy('dissipated')
    Erev[n] = model.getEnergy('reversible')
    Econ[n] = model.getEnergy('cohesive contact')

    # Fragmentation
    
